# 算法实现与模拟实验

### 模拟数据产生器

In [1]:
import math, sys, warnings, traceback
import numpy as np
from data import ar_error_cov1, ar_error_cov2, ar_x_cov, data_generator1
from smrce import training, l1_norm
warnings.filterwarnings("ignore")

In [2]:
matrix_covariance_E1 = ar_error_cov1(6, 0.5)
matrix_precision_E1 = [[int(y*10000)/100.0 for y in x] for x in np.linalg.inv(matrix_covariance_E1).tolist()]
matrix_covariance_E1

[[1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125],
 [0.5, 1.0, 0.5, 0.25, 0.125, 0.0625],
 [0.25, 0.5, 1.0, 0.5, 0.25, 0.125],
 [0.125, 0.25, 0.5, 1.0, 0.5, 0.25],
 [0.0625, 0.125, 0.25, 0.5, 1.0, 0.5],
 [0.03125, 0.0625, 0.125, 0.25, 0.5, 1.0]]

In [3]:
matrix_covariance_E2 = ar_error_cov2(6, 0.95)
matrix_precision_E2 = [[int(y*100)/ 100.0 for y in x] for x in np.linalg.inv(matrix_covariance_E2).tolist()]
matrix_precision_E2

[[4.38, -2.76, -0.28, -0.35, -0.25, -0.41],
 [-2.76, 6.08, -2.6, -0.09, -0.22, -0.25],
 [-0.28, -2.6, 6.09, -2.6, -0.09, -0.35],
 [-0.35, -0.09, -2.6, 6.09, -2.6, -0.28],
 [-0.25, -0.22, -0.09, -2.6, 6.08, -2.76],
 [-0.41, -0.25, -0.35, -0.28, -2.76, 4.38]]

### 交叉评价

In [ ]:
def cross_split(X, Y, n_splits=4):
    D = np.concatenate((X, Y), axis=1)
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=n_splits)
    P = X.shape[1]
    for train, test in kf.split([i for i in range(N)]):
        # training
        D_train, D_test = D[train], D[test]
        X_train, Y_train = D_train[:,:P], D_train[:,P:]
        X_test, Y_test = D_test[:,:P], D_test[:,P:]
        yield X_train, Y_train, X_test, Y_test

### 实验参数组

In [ ]:
params_setup_list = \
[[s1, s2, P, Q, N, roE, n_split, n_repeat] for s1 in [0.1, 0.5] 
 for s2 in [0.5] 
 for P in [20, 50] 
 for Q in [20, 50] 
 for N in [50] 
 for roE in [0.1, 0.3, 0.5, 0.7, 0.9]
 for n_split in [3]
 for n_repeat in [1]]

In [ ]:
for s1, s2, P, Q, N, roE, n_split, n_repeat in params_setup_list:
    
    print("Now, Experiment With Parameters -- %s" % str([s1, s2, P, Q, N, roE]))
    
    #Averaging Hyper Params Out-Sample Performance Using SMRCE 
    
    optimal_lambda_SMRCE = None 
    optimal_rmse_SMRCE = None
    optimal_lambda_RIDGE = None
    optimal_rmse_RIDGE = None
    
    while True:
    
        X, Y, B, CovX = data_generator1(s1, s2, P, Q, N, roE)
    
        for lambda_ in [10, 5, 1, 0.5, 0.1, 0.05, 0.01]:
            print("    Hyper-parameter lambda1 == lambda2 == %3f" % lambda_)

            rmse_sum_SMRCE, rmse_sum_RIDGE = 0, 0

            num = 0
            for i in range(n_repeat):
                for X_tr, Y_tr, X_te, Y_te in cross_split(X, Y, n_split):
                    try:
                        matrix_B_param, B_ridge = training(lambda_, lambda_, X_tr, Y_tr)
                    except:
#                         traceback.print_exc()
                        # Data Set not work well for the hyper-parameter
                        break
                        

                    #Predicting on Out-Sample
                    Y_hat_SMRCE = X_te.dot(matrix_B_param)
                    Y_hat_RIDGE = X_te.dot(B_ridge)

                    rmse_SMRCE = np.sqrt(np.trace((Y_te - Y_hat_SMRCE).T.\
                                            dot(Y_te - Y_hat_SMRCE)))
                    rmse_sum_SMRCE += rmse_SMRCE

                    rmse_RIDGE = np.sqrt(np.trace((Y_te - Y_hat_RIDGE).T.\
                                            dot(Y_te - Y_hat_RIDGE)))
                    rmse_sum_RIDGE += rmse_RIDGE

                    num += 1
            
            if num == 0:
                continue

            avg_rmse_SMRCE = rmse_sum_SMRCE / num
            avg_rmse_RIDGE = rmse_sum_RIDGE / num

            if optimal_rmse_SMRCE is None or optimal_rmse_SMRCE > avg_rmse_SMRCE:
                optimal_rmse_SMRCE = avg_rmse_SMRCE
                optimal_lambda_SMRCE = lambda_

            if optimal_rmse_RIDGE is None or optimal_rmse_RIDGE > avg_rmse_RIDGE:
                optimal_rmse_RIDGE = avg_rmse_RIDGE
                optimal_lambda_RIDGE = lambda_


        # ---- Using Optimal Hyper-parameter Training On the whole Dataset
        #     print(optimal_lambda_SMRCE, optimal_lambda_RIDGE)
        try:
            matrix_B_param, _ = training(optimal_lambda_SMRCE, 
                                         optimal_lambda_SMRCE, 
                                         X, Y)
            _, matrix_B_ridge = training(optimal_lambda_RIDGE, 
                                         optimal_lambda_RIDGE,
                                         X, Y)
        except:
            print("Experiment on this dataset failed!")
            break

        matrix_bias_SMRCE = matrix_B_param - B
        matrix_bias_RIDGE = matrix_B_ridge - B

        element_n = B.shape[0] * B.shape[1]
        mae_bias_SMRCE = l1_norm(matrix_bias_SMRCE) / element_n 
        mae_bias_RIDGE = l1_norm(matrix_bias_RIDGE) / element_n
        rmse_bias_SMRCE = np.sqrt(np.trace(matrix_bias_SMRCE.T.dot(matrix_bias_SMRCE))) / element_n
        rmse_bias_RIDGE = np.sqrt(np.trace(matrix_bias_RIDGE.T.dot(matrix_bias_RIDGE))) / element_n
        l1_elem_SMRCE = l1_norm(matrix_B_param) / element_n
        l1_elem_RIDGE = l1_norm(matrix_B_ridge) / element_n
        l1_elem_B = l1_norm(B) / element_n
        l0_elem_SMRCE = (matrix_B_param != 0).sum()
        l0_elem_RIDGE = (matrix_B_ridge != 0).sum()
        l0_elem_B = (B != 0).sum()

        print(s1, s2, P, Q, N, roE, mae_bias_SMRCE, mae_bias_RIDGE, 
              rmse_bias_SMRCE, rmse_bias_RIDGE,
              l1_elem_SMRCE, l1_elem_RIDGE, l1_elem_B, l0_elem_SMRCE, l0_elem_RIDGE, l0_elem_B)
        break

Now, Experiment With Parameters -- [0.1, 0.5, 20, 20, 50, 0.1]
    Hyper-parameter lambda1 == lambda2 == 10.000000
    Hyper-parameter lambda1 == lambda2 == 5.000000
    Hyper-parameter lambda1 == lambda2 == 1.000000
    Hyper-parameter lambda1 == lambda2 == 0.500000
    Hyper-parameter lambda1 == lambda2 == 0.100000
    Hyper-parameter lambda1 == lambda2 == 0.050000
    Hyper-parameter lambda1 == lambda2 == 0.010000
0.1 0.5 20 20 50 0.1 0.018121967816588983 0.12116977331090428 0.005871356674428618 0.007610094789978464 0.008671892052970618 0.12806305788764857 0.021557431460553764 25 400 11
Now, Experiment With Parameters -- [0.1, 0.5, 20, 20, 50, 0.3]
    Hyper-parameter lambda1 == lambda2 == 10.000000
    Hyper-parameter lambda1 == lambda2 == 5.000000
    Hyper-parameter lambda1 == lambda2 == 1.000000
    Hyper-parameter lambda1 == lambda2 == 0.500000
    Hyper-parameter lambda1 == lambda2 == 0.100000
    Hyper-parameter lambda1 == lambda2 == 0.050000
    Hyper-parameter lambda1 == la

    Hyper-parameter lambda1 == lambda2 == 0.500000
    Hyper-parameter lambda1 == lambda2 == 0.100000
    Hyper-parameter lambda1 == lambda2 == 0.050000
    Hyper-parameter lambda1 == lambda2 == 0.010000
0.1 0.5 50 20 50 0.9 0.029619390167561613 0.11834557386284274 0.005078990316391962 0.00537715741052758 0.00741459752000142 0.11846096055192563 0.035761327584900186 29 1000 52
Now, Experiment With Parameters -- [0.1, 0.5, 50, 50, 50, 0.1]
    Hyper-parameter lambda1 == lambda2 == 10.000000
    Hyper-parameter lambda1 == lambda2 == 5.000000
    Hyper-parameter lambda1 == lambda2 == 1.000000
    Hyper-parameter lambda1 == lambda2 == 0.500000
    Hyper-parameter lambda1 == lambda2 == 0.100000
    Hyper-parameter lambda1 == lambda2 == 0.050000
    Hyper-parameter lambda1 == lambda2 == 0.010000
0.1 0.5 50 50 50 0.1 0.0338945360140826 0.11630617498735328 0.003901765832252259 0.003269237223579125 0.005939618282583346 0.12072653133738313 0.033867816355310984 114 2500 114
Now, Experiment With Pa